<h2>Vamos comparar a Escola de Minas com o ICEB !!!<h3>

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

#importando banco de dado sobre reprovados.
PRO = pd.read_excel('aprovacao_PRO.xlt')
PRO['CURSO'] = 'PRO'
GEO = pd.read_excel('aprovacao_GEO.xlt')
GEO['CURSO'] = 'GEO'
MEC = pd.read_excel('aprovacao_MEC.xlt')
MEC['CURSO'] = 'MEC'
ARQ = pd.read_excel('aprovacao_ARQ.xlt')
ARQ['CURSO'] = 'ARQ'
AMB = pd.read_excel('aprovacao_AMB.xlt')
AMB['CURSO'] = 'AMB'
CAT = pd.read_excel('aprovacao_CAT.xlt')
CAT['CURSO'] = 'CAT'
MET = pd.read_excel('aprovacao_MET.xlt')
MET['CURSO'] = 'MET'
CIV = pd.read_excel('aprovacao_CIV.xlt')
CIV['CURSO'] = 'CIV'
MIN = pd.read_excel('aprovacao_MIN.xlt')
MIN['CURSO'] = 'MIN'

In [2]:
#Excluir observação no dataframe:
PRO = PRO.drop([7055,7056])
#Nas disciplinas do básico não foi possivel separar o total de alunos de Eng de Produção.

#Padronizo o cabeçalho
ARQ = ARQ.rename(columns={"ANO.1": "SEMESTRE","ANO.2": "DISCIPLINA","ANO.3": "DESCRICAO","ANO.4": "DEPARTAMENTO","ANO.5": "TURMA","ANO.6": "APROVACOES","ANO.7": "MATRICULAS","ANO.8": "REPROVACOES FALTA"})

#Excluo colunas nulas.
AMB = AMB.drop(columns=['Unnamed: 12','Unnamed: 13','Unnamed: 14','Unnamed: 15'])
CAT = CAT.drop(columns=['COD PREDIO'])
ARQ = ARQ.drop(columns=['COD PREDIO'])

#Crio a coluna 'DEPARTAMENTO' para o dataframe PRO.
PRO['DEPARTAMENTO'] = PRO['DISCIPLINA']
PRO['DEPARTAMENTO'] = PRO['DEPARTAMENTO'].map(lambda x: str(x)[:-3])
PRO['DE'] = 'DE'
PRO['DEPARTAMENTO'] = PRO['DE'] + PRO['DEPARTAMENTO']
PRO = PRO.drop(columns=['DE'])
PRO['DEPARTAMENTO'] = PRO['DEPARTAMENTO'].str.replace('DEMTM','DEMAT')
PRO['DEPARTAMENTO'] = PRO['DEPARTAMENTO'].str.replace('DEBCC','DECOM')
PRO['DEPARTAMENTO'] = PRO['DEPARTAMENTO'].str.replace('DECIC','DECOM')

In [3]:
#Unindo todas os bancos de dados
Aprovacoes_EM = pd.concat([PRO,AMB,CAT,GEO,CIV,MIN,MET,MEC,ARQ], ignore_index=False)

In [4]:
#excluindo anos indesejados
Aprovacoes_EM['ANO'] = Aprovacoes_EM['ANO'].astype(int)
Aprovacoes_EM = Aprovacoes_EM.loc[(Aprovacoes_EM['ANO'] > 2003)]
Aprovacoes_EM = Aprovacoes_EM.loc[(Aprovacoes_EM['ANO'] != 2020)]

In [5]:
#Retiro um acento do semestre
Aprovacoes_EM['SEMESTRE'] = (Aprovacoes_EM.SEMESTRE.astype(str).str.split('.', expand = True))
#Aprovacoes_EM['SEMESTRE'] = Aprovacoes_EM['SEMESTRE'].astype(int)
Aprovacoes_EM['SEMESTRE'].unique()

array(['1', '2', '3', '4'], dtype=object)

In [6]:
#Vamos unir as colunas Ano e Semestre e deletar o residuo
Aprovacoes_EM['ANO_SEMESTRE'] = Aprovacoes_EM.apply(lambda Aprovacoes_EM:'%s.%s' % (Aprovacoes_EM['ANO'],Aprovacoes_EM['SEMESTRE']),axis=1)
Aprovacoes_EM = Aprovacoes_EM.drop(columns=['ANO','SEMESTRE'])

In [7]:
#Organizando colunas
Aprovacoes_EM = (Aprovacoes_EM[['ANO_SEMESTRE','CURSO','DEPARTAMENTO', 'DISCIPLINA', 'DESCRICAO', 'TURMA','APROVACOES', 'MATRICULAS', 'REPROVACOES FALTA', 'REPROVACOES NOTA','REPROVACOES NOTA FALTA', 'TRANCAMENTOS']]).reset_index()
Aprovacoes_EM = Aprovacoes_EM.drop(columns=['index'])

In [8]:
Aprovacoes_EM['Taxa_aprovação'] = Aprovacoes_EM['APROVACOES']/Aprovacoes_EM['MATRICULAS']
Aprovacoes_EM['Reprovacao_nota'] = Aprovacoes_EM['REPROVACOES NOTA']/Aprovacoes_EM['MATRICULAS']
Aprovacoes_EM['Reprovacao_falta'] = Aprovacoes_EM['REPROVACOES NOTA FALTA']/Aprovacoes_EM['MATRICULAS']

In [9]:
Media_aprovacao_geral = (pd.DataFrame(Aprovacoes_EM.groupby('CURSO').Taxa_aprovação.mean())).reset_index()

In [10]:
Aprovacoes_EM.tail()

,ANO_SEMESTRE,CURSO,DEPARTAMENTO,DISCIPLINA,DESCRICAO,TURMA,APROVACOES,MATRICULAS,REPROVACOES FALTA,REPROVACOES NOTA,REPROVACOES NOTA FALTA,TRANCAMENTOS,Taxa_aprovação,Reprovacao_nota,Reprovacao_falta
57937,2019.2,ARQ,DEMAT,MTM500,FUNDAMENTOS DE CALCULO,32.0,27.0,55.0,1.0,18.0,9.0,0.0,0.490909,0.327273,0.163636
57938,2019.2,ARQ,DEMAT,MTM500,FUNDAMENTOS DE CALCULO,33.0,32.0,49.0,0.0,8.0,8.0,1.0,0.653061,0.163265,0.163265
57939,2019.2,ARQ,DEPRO,PRO562,EFICIENCIA E CONSERVACAO DE ENERGIA,11.0,30.0,34.0,0.0,0.0,0.0,4.0,0.882353,0.000000,0.000000
57940,2019.2,ARQ,DEPRO,PRO562,EFICIENCIA E CONSERVACAO DE ENERGIA,12.0,1.0,1.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000
57941,2019.2,ARQ,DEURB,URB416,GEOTECNIA APLICADA A ARQUITETURA E AO URBANISMO,11.0,29.0,30.0,0.0,0.0,0.0,1.0,0.966667,0.000000,0.000000


Precisamos identificar a Escola de cada matéria.

In [11]:
ICEB = pd.DataFrame({'ESCOLA':'ICEB','DEPARTAMENTO': ['DEFIS','DEMAT','DEQUI','DECOM','DEEST','DEBIO','DECBI']})
EM = pd.DataFrame({'ESCOLA':'ESCOLA DE MINAS', 'DEPARTAMENTO' : ['DECAT','DEMET','DEPRO','DEARQ','DECIV','DEGEO','DEMIN','DEAMB','DEMEC','DEURB']})

In [12]:
Aprovacoes_EM = Aprovacoes_EM.merge(EM,on='DEPARTAMENTO',how = 'outer').merge(ICEB,on='DEPARTAMENTO',how = 'outer')
Aprovacoes_EM

,ANO_SEMESTRE,CURSO,DEPARTAMENTO,DISCIPLINA,DESCRICAO,TURMA,APROVACOES,MATRICULAS,REPROVACOES FALTA,REPROVACOES NOTA,REPROVACOES NOTA FALTA,TRANCAMENTOS,Taxa_aprovação,Reprovacao_nota,Reprovacao_falta,ESCOLA_x,ESCOLA_y
0,2004.1,PRO,DECAT,CAT107,INSTALACOES INDUSTRIAIS,11.0,38.0,38.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,ESCOLA DE MINAS,NaN
1,2004.1,PRO,DECAT,CAT112,EXPRESSAO GRAFICA I,21.0,28.0,29.0,0.0,0.0,1.0,0.0,0.965517,0.000000,0.034483,ESCOLA DE MINAS,NaN
2,2004.1,PRO,DECAT,CAT112,EXPRESSAO GRAFICA I,22.0,15.0,20.0,0.0,0.0,0.0,5.0,0.750000,0.000000,0.000000,ESCOLA DE MINAS,NaN
3,2004.1,PRO,DECAT,CAT113,EXPRESSAO GRAFICA II,21.0,17.0,19.0,0.0,0.0,0.0,2.0,0.894737,0.000000,0.000000,ESCOLA DE MINAS,NaN
4,2004.1,PRO,DECAT,CAT113,EXPRESSAO GRAFICA II,22.0,11.0,11.0,0.0,0.0,0.0,0.0,1.000000,0.000000,0.000000,ESCOLA DE MINAS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57937,2018.2,ARQ,DEURB,URB423,MELHORIAS SANITARIAS EM UNIDADES PREDIAIS SUST...,11.0,11.0,16.0,0.0,0.0,0.0,5.0,0.687500,0.000000,0.000000,ESCOLA DE MINAS,NaN
57938,2019.1,ARQ,DEURB,URB321,SEGURANCA CONTRA INCENDIO DE EDIFICACOES,11.0,12.0,13.0,0.0,0.0,0.0,1.0,0.923077,0.000000,0.000000,ESCOLA DE MINAS,NaN
57939,2019.1,ARQ,DEURB,URB416,GEOTECNIA APLICADA A ARQUITETURA E AO URBANISMO,11.0,28.0,31.0,0.0,1.0,2.0,0.0,0.903226,0.032258,0.064516,ESCOLA DE MINAS,NaN
57940,2019.1,ARQ,DEURB,URB423,MELHORIAS SANITARIAS EM UNIDADES PREDIAIS SUST...,11.0,4.0,7.0,0.0,0.0,0.0,3.0,0.571429,0.000000,0.000000,ESCOLA DE MINAS,NaN


In [13]:
Aprovacoes_EM['ESCOLA_x'] = Aprovacoes_EM['ESCOLA_x'].fillna('')
Aprovacoes_EM['ESCOLA_y'] = Aprovacoes_EM['ESCOLA_y'].fillna('')
Aprovacoes_EM['ESCOLA'] = Aprovacoes_EM['ESCOLA_x'] + Aprovacoes_EM['ESCOLA_y']
Aprovacoes_EM['ESCOLA'] = Aprovacoes_EM['ESCOLA'].fillna('')

In [14]:
Aprovacoes_EM['ESCOLA'] = Aprovacoes_EM['ESCOLA'].replace({'': 'OUTRAS'})
Aprovacoes_EM = Aprovacoes_EM.drop(columns=['ESCOLA_x','ESCOLA_y'])

In [15]:
Aprovacoes_EM.tail()

,ANO_SEMESTRE,CURSO,DEPARTAMENTO,DISCIPLINA,DESCRICAO,TURMA,APROVACOES,MATRICULAS,REPROVACOES FALTA,REPROVACOES NOTA,REPROVACOES NOTA FALTA,TRANCAMENTOS,Taxa_aprovação,Reprovacao_nota,Reprovacao_falta,ESCOLA
57937,2018.2,ARQ,DEURB,URB423,MELHORIAS SANITARIAS EM UNIDADES PREDIAIS SUST...,11.0,11.0,16.0,0.0,0.0,0.0,5.0,0.687500,0.000000,0.000000,ESCOLA DE MINAS
57938,2019.1,ARQ,DEURB,URB321,SEGURANCA CONTRA INCENDIO DE EDIFICACOES,11.0,12.0,13.0,0.0,0.0,0.0,1.0,0.923077,0.000000,0.000000,ESCOLA DE MINAS
57939,2019.1,ARQ,DEURB,URB416,GEOTECNIA APLICADA A ARQUITETURA E AO URBANISMO,11.0,28.0,31.0,0.0,1.0,2.0,0.0,0.903226,0.032258,0.064516,ESCOLA DE MINAS
57940,2019.1,ARQ,DEURB,URB423,MELHORIAS SANITARIAS EM UNIDADES PREDIAIS SUST...,11.0,4.0,7.0,0.0,0.0,0.0,3.0,0.571429,0.000000,0.000000,ESCOLA DE MINAS
57941,2019.2,ARQ,DEURB,URB416,GEOTECNIA APLICADA A ARQUITETURA E AO URBANISMO,11.0,29.0,30.0,0.0,0.0,0.0,1.0,0.966667,0.000000,0.000000,ESCOLA DE MINAS


Devo ler todas as matrizer curriculares e descobrir quais são voluntárias e obrigatórias!!

In [18]:
Grade_MEC = pd.read_excel('matrizMEC.xlsx')
Grade_MEC = (Grade_MEC.dropna()).reset_index().drop(columns = 'index')
Grade_MEC.tail()

,CÓDIGO,DISCIPLINAS OBRIGATÓRIAS
83,MET302,FUNDAMENTOS DE MECANICA DE FRATURA
84,MET303,FADIGA DOS MATERIAIS
85,MTM146,MATEMATICA APLICADA A ENG.DE CONTROLE E\nAUTOM...
86,PRO215,PLANEJAMENTO E CONTROLE DA PRODUCAO I
87,PRO350,PLANEJAMENTO E GESTAO DA QUALIDADE


In [20]:
Grade_MEC['CÓDIGO'].unique().tolist()

['BCC701',
 'MEC100',
 'MEC101',
 'MTM131',
 'MTM700',
 'QUI701',
 'ARQ702',
 'EST202',
 'FIS130',
 'MTM112',
 'MTM702',
 'ARQ703',
 'BCC760',
 'FIS132',
 'MTM125',
 'MTM703',
 'CAT172',
 'FIS134',
 'FIS214',
 'MEC102',
 'MEC119',
 'CAT173',
 'CIV613',
 'MEC103',
 'MEC104',
 'MEC620',
 'MET149',
 'CIV612',
 'MEC105',
 'MEC106',
 'MEC174',
 'MEC416',
 'MEC107',
 'MEC108',
 'MEC110',
 'MEC112',
 'MEC114',
 'AMB111',
 'MEC109',
 'MEC111',
 'MEC113',
 'PRO224',
 'MEC115',
 'MEC117',
 'MEC380',
 'PRO243',
 'DIR133',
 'MEC381',
 'MEC390',
 'PRO244',
 'CÓDIGO',
 'CAT181',
 'CAT305',
 'CAT326',
 'CAT409',
 'CIV500',
 'CIV501',
 'LET966',
 'MEC116',
 'MEC118',
 'MEC339',
 'MEC340',
 'MEC400',
 'MEC401',
 'MEC402',
 'MEC403',
 'MEC404',
 'MEC405',
 'MEC406',
 'MEC407',
 'MEC408',
 'MEC410',
 'MEC411',
 'MEC412',
 'MEC414',
 'MEC415',
 'MEC417',
 'MEC418',
 'MEC419',
 'MEC420',
 'MEC421',
 'MET300',
 'MET301',
 'MET302',
 'MET303',
 'MTM146',
 'PRO215',
 'PRO350']

In [17]:
#descobrindo as eletivas
oi = Grade_MEC
(((oi.loc[oi['CÓDIGO'] == 'CÓDIGO']).index)+1)

Obrigatorias = (Grade_MEC.iloc[[0, 49]]).reset_index().drop(columns = 'index')
Obrigatorias['MODALIDADE'] = 'OBRIGATÓRIA'
Obrigatorias

# Eletivas = (Grade_MEC.iloc[[51,-1]]).reset_index().drop(columns = 'index')
# Eletivas['MODALIDADE'] = 'ELETIVA'
# Eletivas

,CÓDIGO,DISCIPLINAS OBRIGATÓRIAS,MODALIDADE
0,BCC701,PROGRAMACAO DE COMPUTADORES I,OBRIGATÓRIA
1,PRO244,ORGANIZACAO E ADMINISTRACAO II,OBRIGATÓRIA
